# Détection de l'encoding

In [2]:
from bs4 import UnicodeDammit
import chardet

In [23]:
path_serie1 = "/Vrac/PLDAC_addic7ed/data/5122___Blood_(Korean_Drama)/01/02__Episode_2.txt"
rawdata = open(path_serie1, 'rb').read()
result = chardet.detect(rawdata)
charenc = result['encoding']
result

{'confidence': 1.0, 'encoding': 'UTF-8-SIG'}

In [28]:
import os
import re
import nltk
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer

def verifier_ligne(ligne):
    """return True si la ligne est un sous-titre, False sinon"""
    timestamp_regex = r'[0-9]{2}:[0-9]{2}:[0-9]{2}' 
    subnumber_regex =r'^[0-9]+$'
    
    liste_regex = [timestamp_regex, subnumber_regex]

    l = ligne.lower()
    if "addic7ed" in l:
        return False
    #if l.startswith("sync"):
        #return False
    for regex in liste_regex:
        if re.match(regex, ligne):
            return False
    return True

def transformer_ligne(ligne):
    """str -> str
    effectue transformation souhaitees sur la ligne"""
    tag_regex = r'<(/)*[a-zA-Z]+>' #to get rif of tags
    alphanum_regex = r'\W+'  #get rid of non alphanumeric characters
    new_line = re.sub(tag_regex, '', ligne)
    new_line = re.sub(r'\W+', ' ', new_line)
    return new_line

def scan_folder(parent_folder, corp):
    """retourne corpus des textes contenus dans parent_folder sous forme de liste de string"""
    # iterate over all the files in directory 'parent_folder'

    for file_name in sorted(os.listdir(parent_folder)):
        if file_name.endswith(".txt"):
            print(file_name)
            path = parent_folder+"/"+file_name
            rawdata = open(path, 'rb').read()
            result = chardet.detect(rawdata)
            charenc = result['encoding']
            fichier = open(path, "r", encoding = charenc)
            lignes = fichier.readlines()
            fichier.close()
            
            texte = ""
            for ligne in lignes :
                #ligne = ligne.lower()
                if verifier_ligne(ligne):
                    new_line = transformer_ligne(ligne)
                    texte += new_line
            corp.append(texte)
        
        else:
            current_path = "".join((parent_folder, "/", file_name))
            if os.path.isdir(current_path):
                # if we're checking a sub-directory, recall this method
                scan_folder(current_path, corp)
    
    return corp


#pour eviter les variables globales utiliser get_corpus qui appelle scan_folder
def get_corpus(parent_folder):
    """retourne corpus des textes contenus dans parent_folder sous forme de liste de string"""
    c = []
    res = scan_folder(parent_folder, c)
    return res

In [20]:
path_serie1 = "/Vrac/PLDAC_addic7ed/data/175___Avatar__The_Last_Airbender/01"
path_serie2 = "/Vrac/PLDAC_addic7ed/data/1___Lost/01"
corpus = get_corpus(path_serie1)
corpus += get_corpus(path_serie2)
corpus[-1]

'Previously on Lost  Mayday Mayday  The French are coming I ve never been so happy to hear the French  Does anyone speak French  I m alone now  On the island alone  Please someone come  They re dead  It killed them  It killed them all  Guys where are we  I found the wire on the beach  I thought it might have something to do with the transmission we picked up   with a Frenchwoman repeating on a loop for 16 years  Has it really been that long  You  Your people  the ones you re determined to get back to  Watch them closely  I mean I was sleeping and I woke up  and he was trying to hurt my baby   We gotta find out who everyone is  Wanna start a census  I interviewed everyone at the beach Jack  One of them isn t in the manifest  He wasn t on the plane  We re not the only people on this island  and we all know it  They ve attacked us  Sabotaged us abducted us murdered us  Maybe it s time we started worrying about them  Danielle  What are you doing here  When I saw black smoke  That night the

In [29]:
p = "/Vrac/PLDAC_addic7ed/data/"
paths = [p+"886___Judge_John_Deed"]
c = []
for path in paths :
    print(path)
    c += get_corpus(path)

/Vrac/PLDAC_addic7ed/data/886___Judge_John_Deed
00__Exacting_Justice.txt
01__Rough_Justice.txt
02__Duty_of_Care.txt
03__Appropriate_Response.txt
04__Hidden_Agenda.txt
01__Political_Expediency.txt
02__Abuse_of_Power.txt
03__Nobody_s_Fool.txt
04__Everyone_s_Child.txt
01__Health_Hazard.txt
02__Judicial_Review.txt
03__Conspiracy.txt


UnicodeDecodeError: 'charmap' codec can't decode byte 0xfc in position 2789: character maps to <undefined>